# Exploring MNIST with Keras


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/datasets/keras_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/datasets/blob/master/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/datasets/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# Dependencies Block

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers, losses, metrics

print(np.__version__, tf.__version__)

# Loading the Data

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
len(train_images), len(train_labels), \
len(test_images), len(test_labels)

### Exploring the Data

In [ ]:
test_images.shape

In [ ]:
training_labels, training_frequencies = np.unique(train_labels, return_counts = True)
plt.bar(
  training_labels,
  training_frequencies    
)

In [ ]:
testing_labels, testing_frequencies = np.unique(test_labels, return_counts = True)
plt.bar(
  testing_labels,
  testing_frequencies    
)

## Transforming the Data

In [ ]:
def normalize_img(image: np.ndarray):
  """Normalizes images: `uint8` -> `float32`."""
  return image.astype(np.float32) / 255.

normalized_train_images = np.apply_along_axis(normalize_img, 0, train_images)
normalized_test_images =  np.apply_along_axis(normalize_img, 0, test_images)

# Models
## First Starting with a Simple Fully-Connected Feed-Forward Neural Network

In [ ]:
model = models.Sequential([
  layers.Flatten(input_shape=(28, 28)), # this layer converts the 28x28 images (arrays) into 1x784 arrays
  layers.Dense(128, activation='relu'),
  layers.Dense(10) # SIGNAL; not probability predictions
])

model.compile(
    optimizer=optimizers.Adam(0.001),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()], # https://www.tensorflow.org/api_docs/python/tf/keras/metrics/categorical_accuracy
)

model.summary()


In [ ]:
model.fit(
    normalized_train_images, 
    train_labels, 
    epochs=10, 
    validation_data=(normalized_test_images, test_labels)
)

## Then Trying a Convolutional Neural Network

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.compile(
    optimizer=optimizers.Adam(0.001),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[metrics.SparseCategoricalAccuracy()], # https://www.tensorflow.org/api_docs/python/tf/keras/metrics/categorical_accuracy
)

model.summary()

In [ ]:
model.fit(
    normalized_train_images, 
    train_labels, 
    epochs=5, 
    validation_data=(normalized_test_images, test_labels)
)

# Saving the Model!

In [ ]:
model.save("CNN_model")

In [ ]:
new_model = tf.keras.models.load_model('CNN_model')

In [ ]:
new_model.summary()

In [ ]:
!tar -czvf CNN_model.tar.gz CNN_model/